In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
import csv
import string

base_url = "https://newsbook.com.mt/en/category/news/local/page/"
articles = []

# Maltese punctutation
punctuation = string.punctuation.replace('-', '')
punctuation = string.punctuation.replace('\'', '')

for page_num in range(1, 201):
    print(page_num)
    page_url = base_url + str(page_num)
    response = requests.get(page_url)
    soup = bs(response.text, 'lxml')

    links = [a['href'] for a in soup.select('.entry-title a')]

    for link in links:
        try:
            article_response = requests.get(link)
            article_soup = bs(article_response.text, 'html.parser')

            title = article_soup.find('h1', {'class' : 'entry-title'}).text.strip()
            date = article_soup.find('time', {'class' : 'entry-date'}).text.strip()
            body = article_soup.find('div', {'class' : 'td-post-content'})
            paragraphs = body.find_all('p')
            body_cleaned = ' '.join(re.sub(r'[{}]'.format(re.escape(punctuation)), '', p.text).strip() for p in paragraphs if p.text.strip())
            body_cleaned = re.sub(r'\n', ' ', body_cleaned)

            article = {
                'title': title,
                'date': date,
                'body': body_cleaned
            }
            articles.append(article)
        except Exception as e:
            print(f"{e}: {link}")

# Process the scraped articles further if needed
for article in articles:
    print(article)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


KeyboardInterrupt: 

In [ ]:
articles

[{'title': 'Grech: ‘The thieves are turning against each other’',
  'date': 'May 18, 2023 8:54 AM',
  'body': 'Tista taqra bilMalti PN leader Bernard Grech called his supporters on Wednesday as he said that the latest revelations on the fraudulent hospitals deal piloted by former prime minister Joseph Muscat showed that the corrupt are turning against each other Those who thought they were above the law Grech said in reference to Muscat and his former righthand men Keith Schembri and Konrad Mizzi “are hiding and blaming each other” Addressing a protest outside parliament on Wednesday evening Grech said “they are now turning against each other as thieves do” He added that the people “have a mission to give birth to a new Malta a clean Malta where our children can have the future they deserve” Grech recalled that slain journalist Daphne Caruana Galizia had exposed the deceit and corruption in the hospitals deal and said that “we can never stop honouring her” “She gave her life so we can 

In [41]:
import requests
import pandas as pd
import numpy as np
import re
import csv
import string
from bs4 import BeautifulSoup
from io import StringIO

base_url = "https://newsbook.com.mt/en/category/news/local/page/"

# Maltese punctuation
punctuation = string.punctuation.replace('-', '')
punctuation = punctuation.replace('\'', '')
punctuation += '\n'

with open('newsbook_articles.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(["link", "title", "date", "body"])

    # Create a session and connection pool
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    for page_num in range(1, 2500):
        print(page_num)
        page_url = base_url + str(page_num)
        response = session.get(page_url)
        soup = BeautifulSoup(response.text, 'lxml')

        links = [a['href'] for a in soup.select('.entry-title a')]

        # Buffer for accumulating rows
        buffer = StringIO()

        for link in links:
            try:
                article_response = session.get(link)
                article_soup = BeautifulSoup(article_response.text, 'lxml')

                title = article_soup.find('h1', {'class' : 'entry-title'}).text.strip()
                date = article_soup.find('time', {'class' : 'entry-date'}).text.strip()
                body = article_soup.find('div', {'class' : 'td-post-content'})
                paragraphs = body.find_all('p')
                body_text = ' '.join(p.text.strip() for p in paragraphs)
                body_cleaned = re.sub(r'[{}]'.format(re.escape(punctuation)), '', body_text)

                # Accumulate rows in buffer
                buffer.write('"{link}","{title}","{date}","{body_cleaned}"\n'.format(link=link, title=title, date=date, body_cleaned=body_cleaned))

            except Exception as e:
                print(f"{e}: {link}")

        # Write buffered rows to file
        file.write(buffer.getvalue())
        buffer.close()

    # Process the scraped articles further if needed
    for article in articles:
        print(article)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [34]:
punctuation

'!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\n'

In [5]:
with open('newsbook_articles.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["title", "date", "body"])

    for article in articles:
        writer.writerow([article['title'], article['date'], article['body']])